## 前言

**基础知识：**

线性代数、概率论和统计学、最优化理论

Python和Pytorch基础

深度学习基础

## 1. 基本概念

强化学习（Reinforcement Learning，简称RL）是一种机器学习方法，用于让智能系统（如机器人或计算机程序）通过与环境互动来学习如何做出决策，以最大化某种累积奖励信号。强化学习的主要特点是它涉及到智能体（agent）、环境（environment）和奖励信号（reward signal）之间的交互。强化学习的核心任务就是最优策略函数，这个函数可以使用神经网络来拟合，获得更好的性能。

**智能体（Agent）：** 智能体是学习者或决策制定者，它需要学会如何在环境中采取一系列的行动来实现某种目标。智能体通常具有一个决策策略，用于根据观测到的环境状态来选择行动。

**环境（Environment）：** 环境包括智能体所处的外部世界或问题领域。环境的状态可能会随着时间的推移而改变，智能体的行动会影响到环境的演化。

### 1.1 状态

**状态（State）：**状态是指智能体在某一时刻的状态，用S表示。状态可以是离散的，也可以是连续的。例如，船舶智能体在某一时刻的位置、速度、航向等信息，就可以用一个向量来表示，这个向量就是状态。

**离散状态表示：**适用于状态空间较小的情况。例如，在棋盘游戏中，每个棋盘的布局可以被视为一个独特的状态。

$S = \{s_1, s_2, \dots, s_n\}$

**连续状态表示：**对于拥有连续状态空间的环境，如自动驾驶中的位置和速度，通常使用实数向量来表示状态。

$\begin{align*}
\text{船舶动态信息:} \quad & \mathbf{p} = (x, y, z), \quad \mathbf{v} = (v_x, v_y, v_z), \quad \mathbf{a} = (a_x, a_y, a_z), \quad \theta \\
\text{周围环境信息:} \quad & \text{附近船舶位置速度, 导航标识, 声音信号, 气象水文条件} \\
\text{船舶自身状态:} \quad & \text{货仓压力, 温度， 燃油量, 电池电量} \\
\text{传感器数据:} \quad & \text{雷达, AIS, 摄像头数据}
\end{align*}$

**特征表示（Feature Representation）：**将原始状态通过特征提取转换为特征向量。例如，在图像处理的任务中，可以使用卷积神经网络（CNN）提取图像的特征向量作为状态。

$\begin{align*}
\text{船舶动态特性:} \quad & \mathbf{p} = (x, y), \quad \mathbf{v} = (v_x, v_y), \quad 航向角\theta, \quad 转向率r \\
\text{环境因素:} \quad & \text{风速风向, 海流速度方向, 海浪高度周期} \\
\text{航行状态:} \quad & \text{负载状况, 机械系统状态} \\
\text{传感器数据:} \quad & \text{雷达, 声纳数据}
\end{align*}
$

**时间序列表示：**对于需要考虑历史信息的环境，可以使用历史状态序列来表示当前状态，如使用循环神经网络（RNN）处理的序列数据。特别适用于那些需要考虑历史信息以理解当前状态的场景。在时间序列表示中，当前状态不仅依赖于当前的观测，还依赖于过去一系列的观测。

$\mathbf{s}_t = \{s_{t-m}, \dots, s_{t-1}, s_t\}$

**混合表示：** 在某些复杂环境中，可能需要结合以上几种方法来表示状态，以更全面地捕捉环境信息。

$\mathbf{s} = f(s_{\text{离散}}, s_{\text{连续}}, \phi(\text{原始状态}), \{s_{t-m}, \dots, s_t\})$

### 1.2 动作

动作空间表示了智能体（agent）可以采取的所有可能动作的集合。动作空间可以是离散的，也可以是连续的，这取决于具体的应用场景。

**离散动作空间:** 智能体可以选择有限数量的、明确区分的动作。例如，在棋盘游戏中，每次移动一个棋子的动作就构成了离散动作空间。

$A = \{a_1, a_2, \dots, a_m\}$

**连续动作空间:** 在连续动作空间中，动作可以在连续范围内取值。例如，在自动驾驶领域，车辆的转向角度或加速度就是连续动作空间的例子。

$A = [a_{1_{\text{min}}}, a_{1_{\text{max}}}] \times [a_{2_{\text{min}}}, a_{2_{\text{max}}}] \times \cdots \times [a_{n_{\text{min}}}, a_{n_{\text{max}}}]$

### 1.3 奖励函数 R

奖励信号是智能体从环境中获得的反馈，用于评估智能体的行动。奖励通常表示智能体的目标或性能标准，智能体的目标是最大化累积奖励。奖励函数定义了智能体（agent）在特定状态下采取特定动作所获得的即时回报（reward）。奖励函数是智能体学习如何在环境中表现的主要指导，它反映了特定行为的好坏。

$R(s, a, s') = \text{即时奖励}$

### 1.4 回报 G

回报（Return）是指从某一时刻开始到未来某个时刻或无限远的未来，智能体获得的总奖励。回报是评估智能体在一系列时间步骤中表现的关键指标。它通常根据获得的即时奖励和未来奖励的累积来计算。

$G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \cdots = \sum_{k=0}^{\infty} \gamma^k R_{t+k+1}$

注意：

回报（Return）是指单个trajectory获得的discount奖励总和。

状态价值（State Value）是多个trajectory获得回报的期望。

### 1.5 策略 $\pi$

策略（Policy）是智能体（agent）决定如何行动的指导原则。它定义了在给定状态下选择何种动作的概率分布。策略可以是确定性的（Deterministic Policy）或随机性的（Stochastic Policy）。强化学习的目标之一是找到最优策略，即能够最大化累积奖励的策略。

确定性策略:
在确定性策略下，对于每个状态 s，策略 π 选择一个特定的动作 a。用函数形式表示为：$a = \pi(s)$

随机性策略:
在随机性策略下，策略 π 为每个状态和动作对指定一个选择该动作的概率。用函数形式表示为：$\pi(a|s) = P[A = a | S = s]$

### 1.6 值函数 V

值函数（Value Function）：值函数用于估计在给定策略下，智能体在不同状态或状态-行动对下可以获得的预期奖励。值函数可以帮助智能体评估不同策略的好坏。

$V^\pi(s) = \mathbb{E}_\pi[G_t | S = s]$

### 1.7 动作价值函数 

动作值函数（Action-Value Function），通常表示为 Q 函数或 Q 值函数，是在强化学习中用来评估在给定状态下采取不同行动的价值函数。动作值函数用于衡量智能体在特定状态下，采取不同行动后可以获得的预期回报或累积奖励。

$Q^\pi(s, a) = \mathbb{E}_\pi[G_t | S = s, A = a]$

## 2. 动态规划（Dynamic Programming）

动态规划（Dynamic Programming，简称DP）是一种解决优化问题的数学方法，它通常用于解决具有以下两个特征的问题：

1. 重叠子问题（Overlapping Subproblems）：问题可以分解成许多重复性的子问题，这些子问题在解决整体问题时会被多次使用。动态规划的关键思想是将这些子问题的解存储起来，以避免重复计算，从而提高效率。

2. 最优子结构（Optimal Substructure）：问题的最优解可以通过子问题的最优解来构建。这意味着可以通过求解子问题的最优解来找到整体问题的最优解。

动态规划通常包括以下步骤：

1. 确定问题的状态：将问题划分为若干个子问题，并明确定义每个子问题的状态。状态通常用一个或多个变量来表示问题的局部信息。

2. 定义状态转移方程：确定每个子问题之间的关系，即如何从一个子问题的解转移到下一个子问题。这一步骤通常通过递归式或迭代式的方程来完成。

3. 初始化：确定初始状态的值或解。

4. 计算顺序：确定计算子问题的顺序。通常采用自底向上的方式，从最小的子问题开始，逐步计算到整体问题。

5. 存储中间结果：为了避免重复计算，需要将每个子问题的解存储在一个数据结构中，通常使用数组或表格来实现。

6. 求解整体问题：通过计算所有子问题的解，得到整体问题的最优解。

### 2.1 Floyd-Warshall算法

### 2.2 Dijkstra算法

### 2.3 Bellman-Ford算法

### 2.4 0/1背包问题解法

## 3. 马尔科夫决策过程（MDP）

马尔科夫决策过程（Markov Decision Process，简称MDP）是一种用于描述和解决序贯决策问题的数学框架。MDP用于建模在不确定环境中进行决策的问题，其中智能体需要根据其当前状态和可能的行动来选择策略以最大化累积奖励。

马尔科夫过程（Markov Process）： 马尔科夫性通常与马尔科夫过程相关联。马尔科夫过程是一个数学模型，描述了一个随机过程中状态随时间的演变，满足马尔科夫性。具体来说，一个马尔科夫过程在某一时刻 t 的状态转移概率只取决于当前状态，并且与过去的状态历史无关。

马尔科夫链（Markov Chain）： 马尔科夫链是一种特殊类型的马尔科夫过程，它具有离散状态和离散时间步的特征。在马尔科夫链中，状态之间的转移概率只取决于当前状态，不受过去状态序列的影响。马尔科夫链通常用状态转移矩阵来表示，其中每个元素表示从一个状态到另一个状态的转移概率。

$P(S_t = s | S_0, S_1, \ldots, S_{t-1}) = P(S_t = s | S_{t-1})$

### 3.1 贝尔曼方程小例子

贝尔曼方程基于这样一个原理：一个问题的最优解包含其子问题的最优解。在动态规划中，我们将一个大问题分解为相似的小问题，并利用这些小问题的解来构造大问题的解。假设你有一系列决策，每个决策都有其相应的收益和后续状态。贝尔曼方程帮助你找到一系列决策，使得总收益最大化。

例子：假设你正在玩一个简单的游戏，你可以选择向前走一步或两步。每走一步，你会获得一定的分数，目标是最大化总分数。这里的贝尔曼方程可以表达为：

$ V(n) = max(V(n-1) + score(n-1), V(n-2) + score(n-2))$

这个方程的意思是，到达第 n 步的最大分数是由前一步或前两步的最大分数加上当前步的分数中的最大值决定的。

首先，我们会定义一个函数来计算达到每一步的最大分数。这个函数将会使用动态规划的方法，基于之前的步骤来决定当前步骤的最优解。

In [4]:
def max_score(scores):
    # 检查分数列表是否为空
    if not scores:
        return 0

    # 初始化动态规划数组
    dp = [0] * (len(scores) + 1)

    # 初始化前两步的分数
    dp[1] = scores[0]
    if len(scores) > 1:
        dp[2] = scores[0] + scores[1]

    # 动态规划计算每一步的最大分数
    for i in range(3, len(scores) + 1):
        dp[i] = max(dp[i - 1] + scores[i - 1], dp[i - 2] + scores[i - 2])

    # 返回最后一步的最大分数
    return dp[-1]

# 测试例子
scores = [5, 6, 7, 8, 9]
print("最大可获得的分数是:", max_score(scores))


最大可获得的分数是: 35


这段代码首先进行了输入的基本检查，然后使用了一个数组 dp 来存储到达每一步时可能获得的最大分数。它通过遍历每一步，并选择从前一步或前两步迈向当前步时能得到的最大分数来更新这个数组。测试用例使用了一个简单的分数列表 [5, 6, 7, 8, 9]。

### 3.2 贝尔曼方程

在强化学习中，贝尔曼方程提供了一种递归的方式来计算状态的价值或者某个策略的价值。

#### 状态价值函数 $V^{\pi}(s)$

$V^{\pi}(s) = \mathbb{E}_{\pi}\left[\sum_{k=0}^{\infty} \gamma^{k} R_{t+k+1} \mid S_t = s\right]$

其中：

$V^{\pi}(s)$表示在策略$\pi$下状态$s$的价值。

$\mathbb{E}_\pi$表示在策略$\pi$下的期望值。

$\gamma$是折扣因子，范围在 0 到 1 之间，用于减少未来奖励的影响。

$R_{t+k+1}$是在时刻$t+k+1$获得的奖励。

$S_t$是在时刻 $t$ 的状态。


**贝尔曼方程为状态价值函数提供了一个递归的表达方式：**

$V^{\pi}(s) = \sum_{a \in A} \pi(a|s) \sum_{s', r} p(s', r | s, a) [r + \gamma V^{\pi}(s')]$

其中：

$\pi(a|s)$ 是在状态$s$下采取动作$a$的策略概率。

$p(s', r | s, a)$ 是从状态$s$ 采取动作$a$到达状态$s'$并获得奖励$r$的概率。

$r$ 和 $s'$ 分别是获得的即时奖励和转移到的新状态。

**注意：**

状态价值函数 $V^{\pi}(s)$提供了评估策略 $\pi$的一个方法，通过计算在该策略下从特定状态$s$开始所能获得的预期总回报。通过贝尔曼方程，我们可以看到状态价值不仅取决于当前状态和动作，还取决于后续状态的价值。在实际应用中，强化学习算法通常会尝试估计或近似这个状态价值函数，以指导智能体的决策过程。


#### 动作价值函数 $Q^{\pi}(s, a)$

动作价值函数（Action-Value Function），也称为 Q 函数，是强化学习中的一个基本概念，用于评估在给定状态下采取特定动作的预期回报。

$Q^{\pi}(s, a) = \mathbb{E}_{\pi}\left[\sum_{k=0}^{\infty} \gamma^{k} R_{t+k+1} \mid S_t = s, A_t = a\right]$

其中：

$Q^{\pi}(s, a)$ 表示在策略$\pi$下，在状态$s$采取动作$a$的价值。

$\mathbb{E}_\pi$表示在策略$\pi$下的期望值。

$\gamma$ 是折扣因子，范围在 0 到 1 之间，用于减少未来奖励的影响。

$R_{t+k+1}$是在时刻$t+k+1$获得的奖励。

$S_t$和$A_t$分别是在时刻 $t$ 的状态和采取的动作。

**贝尔曼方程为动作价值函数提供了一个递归的表达方式：**

$Q^{\pi}(s, a) = \sum_{s', r} p(s', r | s, a) [r + \gamma \sum_{a' \in A} \pi(a'|s') Q^{\pi}(s', a')]$

其中：

$p(s', r | s, a)$ 是从状态$s$ 采取动作$a$到达状态$s'$并获得奖励$r$的概率。

$\sum_{a' \in A} \pi(a'|s') Q^{\pi}(s', a')$ 表示在下一个状态$S'$下，根据策略$\pi$选择所有可能动作 $a'$的期望价值。

动作价值函数 $Q^{\pi}(s, a)$提供了评估策略$\pi$的一个方法，通过计算在该策略下从特定状态$s$开始并采取特定动作$a$所能获得的预期总回报。
贝尔曼方程表明，动作价值不仅取决于当前状态和动作，还取决于后续状态和动作的价值。
在强化学习的应用中，算法通常会尝试估计或近似这个动作价值函数，以指导智能体的行为决策。

### 3.3 贝尔曼最优公式

#### 贝尔曼最优状态价值函数

贝尔曼最优状态价值函数描述了在最优策略下，某状态的最大预期回报。它定义为：

$V^*(s) = \max_a \sum_{s', r} p(s', r | s, a) [r + \gamma V^*(s')]$

其中：

$V^*(s)$ 是在状态$s$下最优策略的价值。

$max_a$ 表示对所有可能的动作进行最大化。

$p(s', r | s, a)$ 是在状态$s$下采取动作$a$后，转移到状态$s'$并接收到奖励$r$的概率

$\gamma$ 是折扣因子，用于减少未来奖励的权重。

#### 贝尔曼最优动作价值函数

贝尔曼最优动作价值函数描述了在最优策略下，某状态采取某动作的最大预期回报。它定义为：

$Q^*(s, a) = \sum_{s', r} p(s', r | s, a) [r + \gamma \max_{a'} Q^*(s', a')]$

其中：

$Q^*(s, a)$ 是在状态$s$下采取动作 $a$在最优策略下的价值。

$\sum_{s', r} $ 表示对所有可能的下一个状态$s'$和奖励$r$的求和。

$\max_{a'}$表示对所有可能的下一个动作$a'$进行最大化。

## 4. 时序差分（TD）

#### 时序差分

时序差分（Temporal Difference, TD）学习是一种在强化学习领域中非常重要的方法，它结合了蒙特卡洛方法和动态规划的思想。TD学习的关键特点在于它如何估计和更新价值函数（例如状态价值或动作价值），这些价值函数反映了在给定策略下从特定状态或状态-动作对开始的预期回报。

**TD学习的主要特点包括：**

1. 基于差分更新：在TD学习中，价值函数的更新是基于当前估计和实际观察到的回报之间的差异（即“时序差分”错误）。

2. 自举（Bootstrapping）：TD方法通过部分依赖于现有价值估计来更新其预测。这种自举方法意味着TD算法不需要等待最终结果就可以进行学习，与蒙特卡洛方法相比，这是一个显著的不同点。

3. 在线更新：TD学习可以在从环境中获取每一个经验（例如状态转换）后进行更新，不需要等待整个序列结束。这使得TD学习非常适合于连续的、没有明确终点的任务。

4. 策略评估和控制：TD学习既可以用于策略评估（即估计给定策略的价值），也可以用于策略控制（即寻找最优策略）。在策略控制中，常见的TD学习方法有Q学习和SARSA。

$
\begin{align*}
\text{在时序差分 (TD) 学习中, 基本更新规则如下:} & \\
V(S_t) &\leftarrow V(S_t) + \alpha [R_{t+1} + \gamma V(S_{t+1}) - V(S_t)] \\
\text{其中:} & \\
V(S_t) &\text{ 是在时间 } t \text{ 的状态 } S_t \text{ 的价值估计。} \\
\alpha &\text{ 是学习率, 决定了新信息覆盖旧信息的速度。} \\
R_{t+1} &\text{ 是在时间 } t \text{ 到 } t+1 \text{ 之间获得的奖励。} \\
\gamma &\text{ 是折扣因子, 用于调节未来奖励的重要性。} \\
V(S_{t+1}) &\text{ 是下一个状态 } S_{t+1} \text{ 的价值估计。}
\end{align*}
$

#### 蒙特卡洛方法

蒙特卡洛方法是一种基于随机样本来解决各种计算问题的方法。在强化学习领域，蒙特卡洛方法特指用于估计和优化策略的价值函数的技术。这些方法通过分析经历的完整序列（如一局游戏的开始到结束）来学习，而不是依赖于模型预测或部分序列。

特卡洛方法需要等到一个序列（episode）结束后，才根据整个序列获得的总回报来更新价值估计。这种方法适用于有明确起点和终点的情景，如棋类游戏。它不依赖于对环境动态的了解，即不需要知道状态转移概率和奖励函数，直接通过经验来学习价值函数或策略。蒙特卡洛方法可以用于策略评估（估计给定策略下的状态或动作价值）和策略优化（找到最优策略）。蒙特卡洛方法通常依赖于探索性的初始策略来确保覆盖所有可能的状态或动作。这是因为它们的学习完全基于从实际经验中获取的样本。与时序差分方法不同，蒙特卡洛方法在更新价值估计时不依赖于现有的价值估计，而是直接使用实际回报。

缺点：由于基于完整序列的回报来更新价值估计，蒙特卡洛方法可能会有较高的方差，尤其是在序列长度变化很大的情况下。

**TD算法和蒙特卡洛方法的共同点和不同点**

相同点：

1. 基于样本的更新：蒙特卡洛方法和TD算法都使用从环境中采集的样本（即状态转换、奖励等）来更新价值估计。这与基于完整环境模型的动态规划方法不同，动态规划需要知道所有可能的状态转换和奖励。

2. 无需环境模型：两者都不需要事先知道环境的完整模型，这使得它们适用于更广泛的、特别是模型未知的场景。

3. 对策略的依赖：TD学习和蒙特卡洛方法都可以用于评估给定策略下的状态价值，或者在控制问题中寻找最优策略。

4. 使用奖励信号：它们都直接使用从环境中获得的奖励来更新价值估计。

不同之处：

1. 更新时机：蒙特卡洛方法需要等到一个完整的序列（如一局游戏的结束）完成后，才能根据序列中获得的总回报来更新价值估计。而TD学习可以在每个时间步进行更新，不需要等待序列结束。

2. 估计的基础：蒙特卡洛方法基于实际回报来估计价值，而TD学习则使用当前的价值估计来预测未来的价值，这就是所谓的自举（bootstrapping）。



### 4.1 SARSA（State-Action-Reward-State-Action）

SARSA算法是一种在强化学习中用于策略控制的时序差分（Temporal Difference, TD）学习方法。它属于一种在线策略学习算法，即在学习过程中，它根据当前策略来决定其行动。SARSA名称来自于算法使用的五个数据元素：状态（State），动作（Action），奖励（Reward），下一个状态（Next State），以及在此状态下采取的下一个动作（Next Action）。

SARSA算法的更新规则可以使用以下 LaTeX 公式表示：

$Q(S_t, A_t) \leftarrow Q(S_t, A_t) + \alpha [R_{t+1} + \gamma Q(S_{t+1}, A_{t+1}) - Q(S_t, A_t)]$

### 4.2 Q学习（Q-Learning）

### 4.3 TD(λ)（Temporal Difference Learning with Eligibility Traces）

### 4.4 Deep Q-Network（DQN）

### 4.5 Actor-Critic （A2C、A3C、DDPG、PPO)